In [ ]:
import antigravity

In [ ]:
import this

# Multiprocessing and Multithreading

###### Goal
<ul>
<li>Speed up code by using multiple processes</li>
</ul>

###### Options
<ul>
<li>Multithreading</li>
<li>Multiprocessing</li>
</ul>

#### Multithreading

Can use when
<ul>
<li>Lots time waiting around for a response</li>
<ul>
<li>Network Requests - http get, post, put</li>
</ul>
<li>
Lots of I/O (Read, Write, Send, Recv...)
</li>
</ul>

##### Still Bound by Global Interpreter Lock

## CPU Bound Threading

In [ ]:
from __future__ import division
from threading import Thread
import time

Make a list with 10 million 10's.

In [ ]:
len_x = 10000000
x = [10]*len_x
x[:15]

Squaring Function

In [ ]:
def squared(num):
    num**2

In [ ]:
def squareList(nums):
    for i in nums:
        squared(i)

### Square all of the numbers in the list.

###### No threading

In [ ]:
start = time.time() #  Get current time
squareList(x)
serialprocesstime = time.time() - start
print("Serial computation took {} seconds.".format(serialprocesstime))

###### With threads

give each thread 1/4th of the work

In [ ]:
x2 = [10]*(len_x//4)
t1 = Thread(target=squareList, args=(x2,))
t2 = Thread(target=squareList, args=(x2,))
t3 = Thread(target=squareList, args=(x2,))
t4 = Thread(target=squareList, args=(x2,))

start = time.time()
t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print("4 threads took {} seconds.".format(time.time() - start))

We can also use the threadpool

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool

In [ ]:
pool = ThreadPool(4)
start = time.time()
results = pool.map(squared, x)
print("4 thread pool took {} seconds".format(time.time() - start))

## I/O Bound Threading

open up some webpages

In [ ]:
import urllib2

In [ ]:
webpages = ["http://lifehacker.com/", "https://uoregon.edu/", "http://www.goducks.com/", "https://docs.python.org/2/howto/urllib2.html"]

In [ ]:
def visit(url):
    return urllib2.urlopen(url)

### Without threads

In [ ]:
start = time.time()
for url in webpages:
    visit(url)
print("Serial method took {} second to open 4 webpages.".format(time.time() - start))

### With threads

In [ ]:
t1 = Thread(target=visit, args=(webpages[0],))
t2 = Thread(target=visit, args=(webpages[1],))
t3 = Thread(target=visit, args=(webpages[2],))
t4 = Thread(target=visit, args=(webpages[3],))

start = time.time()
t1.start()
t2.start()
t3.start()
t4.start()
t1.join()
t2.join()
t3.join()
t4.join()
print("4 thread pool took {} seconds to open 4 webpages.".format(time.time() - start))

Threads can make I/O bound programs significantly faster.

Note:  I noticed that the threading actually ran slower if I wasn't connected with an ethernet cable at home.

# Squaring with multiprocessing

Let's do the same squaring function we did before, but this time with multiprocessing

In [ ]:
from multiprocessing import Process
import time

In [ ]:
len(x)

According to the python docs, pool cannot be used in the interactive interpreter.

It seems that this extends to an ipython notebook.
<a href="https://docs.python.org/3/library/multiprocessing.html#using-a-pool-of-workers">python pool docs</a>

In [ ]:
num_pros = 4

pro_list = []
x3 = [10]*(len_x//num_pros)
for p in range(num_pros):
    p = Process(target=squareList, args=(x3,))
    pro_list.append(p)

start = time.time()
for p in pro_list:
    p.start()

for p in pro_list:
    p.join()

squareprocesstime = time.time() - start
print("Squaring 10 million numbers took {} seconds with 4 processes.".format(squareprocesstime))

### Note:
When I ran this code on windows running these processes resulted in drastic speed decreases (30x).  But running the same code on ubuntu 15.10 works fine.  I'm still not sure why.  

Speed increase over serial method

In [ ]:
squareprocesstime/serialprocesstime

# Locks

When multiple processes try to access a single resource or variable, read and writes can overlap causing problems, so locks are necessary.

In [ ]:
from multiprocessing import Lock, Value

In [ ]:
lock = Lock()
counter = Value("i", 0)

def noLockCount():
    global counter
    for i in xrange(10000):
        counter.value += 1
    return counter

def lockCounter(lock):
    global counter
    
    with lock:
        for i in xrange(10000):
            counter.value += 1
    
    return counter

Here is analogous code for lockCounter

In [ ]:
def sameLockCounter(lock):
    global counter
    
    lock.acquire()
    for i in xrange(10000):
        counter.value += 1
    lock.release()
    
    return counter

### No Lock

If we run 4 processes and add 1 to the counter 10,000 times each, we should expect to see 40,000 as our output

In [ ]:
counter = Value("i", 0)
num_processes = 4
p_list = []

for i in range(num_processes):
    p = Process(target=noLockCount, args=())
    p_list.append(p)
    
for p in p_list:
    p.start()
    
for p in p_list:
    p.join()
    
print(counter.value)

### With Lock

In [ ]:
counter = Value("i", 0)
num_processes = 4
p_list = []

for i in range(num_processes):
    p = Process(target=lockCounter, args=(lock,))
    p_list.append(p)
    
for p in p_list:
    p.start()
    
for p in p_list:
    p.join()
    
print(counter.value)

### Concluding Thoughts

Just starting to use multiprocessing -> stick with threads unless you are cpu bound

-> use multiprocessing.dummy.ThreadPool.  It is the cleanest and easiest